# CAPSTONE PROJECT

## Introduction: Business Problem

In this project, we want to find out which factor affects on the housing crowded in Chicago. The factors are mentioned by crime rate, number of shops and restaurants, number of schools, distance to the center city. First we have to take data from some sources and clean, normalize. Second, we use the visualization to see the correlation between the percent of housing crowded and each factors. Then we decide which factors are choosen to be the features. Finally by using Regression model, we look for coefficient of each factor and see which is the factor having the most affection of the housing crowded.

## Data

To explore our problem, the date is required some information:
* The rate of crimes in neighborhood
* The number of shops and restaurants in neighborhood
* The number of schools
* The distance of neighborhood to the center city

To get the data, we can take 
* the list  of neighborhoods in Chicago from https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv
* the list of crime in Chicago from https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv
* The number of shop and restaurant in every neighborhood by using  **Foursquare API**
* the number of schools in https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv
* coordinate of Chicago center will be obtained using geolocator function

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors



#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
data_neighbors=pd.read_csv("https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv")
data_neighbors.dtypes

COMMUNITY_AREA_NUMBER                           float64
COMMUNITY_AREA_NAME                              object
PERCENT OF HOUSING CROWDED                      float64
PERCENT HOUSEHOLDS BELOW POVERTY                float64
PERCENT AGED 16+ UNEMPLOYED                     float64
PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA    float64
PERCENT AGED UNDER 18 OR OVER 64                float64
PER_CAPITA_INCOME                                 int64
HARDSHIP_INDEX                                  float64
dtype: object

In [3]:
data_neighbors.shape

(78, 9)

In the data_neighbors, we take only the two columns "COMMUNITY_AREA_NAME" and "PERCENT OF HOUSING CROWDED".

In [4]:
data=data_neighbors[['COMMUNITY_AREA_NUMBER','COMMUNITY_AREA_NAME','PERCENT OF HOUSING CROWDED']]
data.head()

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT OF HOUSING CROWDED
0,1.0,Rogers Park,7.7
1,2.0,West Ridge,7.8
2,3.0,Uptown,3.8
3,4.0,Lincoln Square,3.4
4,5.0,North Center,0.3


We download the list of crime and count the number of crimes followed the neighbors. 

In [3]:
chicago_crime= pd.read_csv('https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv')
chicago_crime.head()

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATEDON,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,08/28/2004 05:50:56 PM,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,911,9,14.0,58.0,6,1155838.0,1873050.0,2004,02/10/2018 03:50:01 PM,41.807441,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,06/26/2004 12:40:00 PM,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,1112,11,27.0,23.0,6,1152206.0,1906127.0,2004,02/28/2018 03:56:25 PM,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,04/04/2011 05:45:00 AM,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,221,2,3.0,38.0,6,1177436.0,1876313.0,2011,02/10/2018 03:50:01 PM,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,12/30/2010 04:30:00 PM,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,423,4,7.0,46.0,6,1194622.0,1850125.0,2010,02/10/2018 03:50:01 PM,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,02/02/2016 07:30:00 PM,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,831,8,15.0,66.0,6,1155240.0,1860661.0,2016,02/10/2018 03:50:01 PM,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [80]:
chicago_crime.LATITUDE

0      41.807441
1      41.898280
2      41.815933
3      41.743665
4      41.773455
5      41.896447
6      41.972845
7      41.823880
8      41.711281
9      41.939296
10     41.754593
11     41.865761
12     41.697154
13     41.868243
14     41.872584
15     41.929744
16     41.732731
17     41.750772
18     41.875659
19     41.869174
20     41.897983
21     41.881598
22     41.769204
23     41.827374
24     41.785123
25     41.961845
26     41.976290
27     41.773204
28     41.704455
29     41.750102
30     41.879104
31     41.828502
32     41.939539
33     41.810834
34     41.853725
35     41.925114
36     41.896140
37     41.898676
38     41.768331
39     41.903142
40     41.882852
41     41.763467
42     41.852275
43     41.913290
44     41.943167
45     41.779794
46     41.929240
47     41.880653
48     41.911798
49     41.936046
50     41.793441
51     41.775793
52     41.881407
53     41.854811
54     41.901488
55     41.889453
56     41.911993
57     41.873957
58     41.9959

In [7]:
venues_map = folium.Map(location=[CA_latitude,CA_longitude], zoom_start=13) # generate map centred around the Conrad Hotel



for lat, lng in zip(chicago_crime.LATITUDE, chicago_crime.LONGITUDE):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        fill=True,
        color='blue',
        fill_color='blue',
        ).add_to(venues_map)

# display map
venues_map

ValueError: Location values cannot contain NaNs, got:
[nan, nan]

In [6]:
df=chicago_crime['COMMUNITY_AREA_NUMBER'].value_counts()
data_crime=pd.DataFrame({'COMMUNITY_AREA_NUMBER':df.index,'number_crimes':df.values})
data_crime.head()

,COMMUNITY_AREA_NUMBER,number_crimes
0,25.0,43
1,23.0,22
2,68.0,21
3,29.0,16
4,28.0,16


Add data_crime to data

In [7]:
data=pd.merge(data, data_crime, on='COMMUNITY_AREA_NUMBER')
data.head()

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT OF HOUSING CROWDED,number_crimes
0,1.0,Rogers Park,7.7,6
1,2.0,West Ridge,7.8,7
2,3.0,Uptown,3.8,4
3,4.0,Lincoln Square,3.4,3
4,5.0,North Center,0.3,4


In [5]:
#Take the latitude and longtitude of Chicago center
address = 'Chicago, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
CA_latitude = location.latitude
CA_longitude = location.longitude
CA_coords=(CA_latitude,CA_longitude)
print('The geograpical coordinate of Chicago City are {}, {}.'.format(CA_latitude, CA_longitude))

The geograpical coordinate of Chicago City are 43.58919945, -79.6489756825067.


In [15]:
data=data.drop(64)

In [16]:
data.set_index(['COMMUNITY_AREA_NUMBER'],inplace=True)
data.head()

,COMMUNITY_AREA_NAME,PERCENT OF HOUSING CROWDED,number_crimes
COMMUNITY_AREA_NUMBER,,,
1.0,Rogers Park,7.7,6
2.0,West Ridge,7.8,7
3.0,Uptown,3.8,4
4.0,Lincoln Square,3.4,3
5.0,North Center,0.3,4


In [17]:
data.reset_index(inplace=True)

In [18]:
data

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT OF HOUSING CROWDED,number_crimes
0,1.0,Rogers Park,7.7,6
1,2.0,West Ridge,7.8,7
2,3.0,Uptown,3.8,4
3,4.0,Lincoln Square,3.4,3
4,5.0,North Center,0.3,4
5,6.0,Lake View,1.1,11
6,7.0,Lincoln Park,0.8,5
7,8.0,Near North Side,1.9,15
8,10.0,Norwood Park,2.0,3
9,11.0,Jefferson Park,2.7,2


Add the distance between Chicago center and each neighbor on data_neighbors

In [24]:

from geopy.distance import geodesic
ls_distance=[]
ls_latitude=[]
ls_longitude=[]
for i in range(data.shape[0]):
    
    address = data['COMMUNITY_AREA_NAME'][i]+ ',CA'

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    Nei_latitude = location.latitude
    Nei_longitude = location.longitude
    Nei_coords=(Nei_latitude,Nei_longitude)
    distance=geodesic(CA_coords, Nei_coords)
    d=str(distance)
    d_new=d[:-3]
    d_new=float(d_new)
    ls_distance.append(d_new)
    ls_latitude.append(Nei_latitude)
    ls_longitude.append(Nei_longitude)
data['Distance']=ls_distance
data['Latitude']=ls_latitude
data['Longitude']=ls_longitude

In [25]:
data.dtypes

COMMUNITY_AREA_NUMBER         float64
COMMUNITY_AREA_NAME            object
PERCENT OF HOUSING CROWDED    float64
number_crimes                   int64
Distance                      float64
Latitude                      float64
Longitude                     float64
dtype: object

In [26]:
data.head()

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT OF HOUSING CROWDED,number_crimes,Distance,Latitude,Longitude
0,1.0,Rogers Park,7.7,6,3493.627660,33.968495,-118.360999
1,2.0,West Ridge,7.8,7,3384.027934,34.287280,-117.213938
2,3.0,Uptown,3.8,4,3618.221568,37.809276,-122.269947
3,4.0,Lincoln Square,3.4,3,3618.514957,37.800252,-122.268544
4,5.0,North Center,0.3,4,1202.377008,46.109254,-64.832244


Now we will add the number of shops and restaurants into data

In [43]:
client_id = 'YQYJICCBDPMQ5B3AV2QIBLIGHV2WP0AVBKDP2BGNTH41R1YB' # your Foursquare ID
client_secret = 'IRPMEG1XPDMEUZFUJPH514KV32K3F1NFFKG13Y2OJ4FKQ5XR' # your Foursquare Secret
version = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: YQYJICCBDPMQ5B3AV2QIBLIGHV2WP0AVBKDP2BGNTH41R1YB
CLIENT_SECRET:IRPMEG1XPDMEUZFUJPH514KV32K3F1NFFKG13Y2OJ4FKQ5XR


In [77]:
radius=50000
category='4bf58dd8d48988d104941735'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(client_id, client_secret,version, CA_latitude,CA_longitude, category,radius,100)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d25d8cea6ec98002c72793e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b1d38d8f964a520360d24e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d104941735',
         'name': 'Medical Center',
         'pluralName': 'Medical Centers',
         'primary': True,
         'shortName': 'Medical'}],
       'id': '4b1d38d8f964a520360d24e3',
       'location': {'address': '82 Buttonwood Ave.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Jane and weston rd.',
        'distance': 15789,
        'formattedAddress': ['82 Buttonwood Ave. (Jane and weston rd.)',
         'Toronto ON',
         'Canada'],
      

In [78]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4b1d38d8f964a520360d24e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d104941735',
    'name': 'Medical Center',
    'pluralName': 'Medical Centers',
    'primary': True,
    'shortName': 'Medical'}],
  'id': '4b1d38d8f964a520360d24e3',
  'location': {'address': '82 Buttonwood Ave.',
   'cc': 'CA',
   'city': 'Toronto',
   'country': 'Canada',
   'crossStreet': 'Jane and weston rd.',
   'distance': 15789,
   'formattedAddress': ['82 Buttonwood Ave. (Jane and weston rd.)',
    'Toronto ON',
    'Canada'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.68806439521652,
     'lng': -79.50843099683723}],
   'lat': 43.68806439521652,
   'lng': -79.50843099683723,
   'state': 'ON'},
  'name': 'West Park Healt

In [79]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.shape

(100, 16)